In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fewshott5/data_process.py
/kaggle/input/fewshott5/layers.py
/kaggle/input/fewshott5/t5.py
/kaggle/input/fewshott5/model.py
/kaggle/input/fewshott5/model_v2.py
/kaggle/input/fewshott5/discriminator.py
/kaggle/input/fewshott5/utils.py
/kaggle/input/fewshott5/__init__.py
/kaggle/input/fewshott5/.ipynb_checkpoints/discriminator-checkpoint.py
/kaggle/input/fewshott5/.ipynb_checkpoints/t5-checkpoint.py
/kaggle/input/fewshott5/.ipynb_checkpoints/data_process-checkpoint.py
/kaggle/input/fewshott5/.ipynb_checkpoints/__init__-checkpoint.py
/kaggle/input/fewshott5/.ipynb_checkpoints/layers-checkpoint.py
/kaggle/input/fewshott5/.ipynb_checkpoints/model-checkpoint.py
/kaggle/input/fewshott5/.ipynb_checkpoints/utils-checkpoint.py
/kaggle/input/fewshott5/.ipynb_checkpoints/model_v2-checkpoint.py
/kaggle/input/fewshott5/Data/csl_title/dataset_dict.json
/kaggle/input/fewshott5/Data/csl_title/train/state.json
/kaggle/input/fewshott5/Data/csl_title/train/dataset_info.json
/kaggle/input/fews

In [2]:
os.sys.path.append('/kaggle/input/')

In [3]:
!pip install rouge sacrebleu
# !pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 603.0 kB/s eta 0:00:00


In [4]:
import os
import datasets
import torch
import tqdm
import numpy as np
from fewshott5.model_v2 import T5ForFewshotGeneration
from fewshott5.utils import set_seed
from fewshott5.data_process import MyDataset

In [5]:
class CFG:
    model_name_or_path = "Langboat/mengzi-t5-base"
    output_dir = "./outputs/"
    is_zh = True
    num_saves_kept = 3
    max_source_length = 512
    max_target_length = 41
    num_beams = 4
    train_batch_size = 1
    eval_batch_size = 12
    num_eval_batch = 50
    gradient_accumulation_steps = 2
    pretrain_steps = 50
    gold_steps = 5
    learning_rate = 5e-5
    weight_decay = 0.01
    adam_epsilon = 1e-6
    max_grad_norm = 2.0
    num_train_epochs = 3
    max_train_steps = 5000
    warmup_ratio = 0.
    warmup_steps = 0
    lambda_idt = 0.1
    lambda_cyc = 0.2
    logging_steps = 25
    save_steps = 100
    seed = 2022
    scheduler_type = "cosine"
    use_amp = True
    scale_loss = 2 ** 15
    logdir = "logs"
    num_workers = 4
    ckp_path = None
    init_step = 0

CFG.logdir = os.path.join(CFG.output_dir, CFG.logdir)
# if not os.path.exists(CFG.model_name_or_path):
#     CFG.model_name_or_path = "t5-base"
set_seed(CFG.seed)

In [6]:
model = T5ForFewshotGeneration(CFG)

Downloading:   0%|          | 0.00/708k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/659 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/945M [00:00<?, ?B/s]

In [7]:
train_dataset = MyDataset(model.tkn, model.args.train_batch_size, "../input/fewshott5/Data/csl_title", ['abst', 'title'], split='train')
dev_dataset = MyDataset(model.tkn, model.args.eval_batch_size, "../input/fewshott5/Data/csl_title", ['abst', 'title'], split='dev')

In [8]:
model.finetune(train_dataset, dev_dataset, True)

[2022-05-28 15:55:08,432][model_v2.py][line:426][INFO] ********** Running training **********
[2022-05-28 15:55:08,434][model_v2.py][line:426][INFO]   Num examples = 3000
[2022-05-28 15:55:08,437][model_v2.py][line:426][INFO]   Num Epochs = 4
[2022-05-28 15:55:08,440][model_v2.py][line:426][INFO]   Instantaneous train batch size = 1
[2022-05-28 15:55:08,441][model_v2.py][line:426][INFO]   Instantaneous eval batch size = 12
[2022-05-28 15:55:08,441][model_v2.py][line:426][INFO]   Total train batch size (w. accumulation) = 2
[2022-05-28 15:55:08,442][model_v2.py][line:426][INFO]   Gradient Accumulation steps = 2
[2022-05-28 15:55:08,445][model_v2.py][line:426][INFO]   Total optimization steps = 5000


********** Running training **********
  Num examples = 3000
  Num Epochs = 4
  Instantaneous train batch size = 1
  Instantaneous eval batch size = 12
  Total train batch size (w. accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 5000


[2022-05-28 15:55:08,643][arrow_dataset.py][line:2785][WARNING] Loading cached shuffled indices for dataset at ../input/fewshott5/Data/csl_title/train/cache-f1e62f35a1a86843.arrow
[2022-05-28 15:55:08,653][model_v2.py][line:426][INFO] ********** Pretraining **********


********** Pretraining **********


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
[2022-05-28 15:56:06,791][model_v2.py][line:426][INFO] ********** Pretraining finished**********


...

{'rouge-1': 0.4516643544176362, 'rouge-2': 0.32954916535989054, 'rouge-l': 0.40849861773713314, 'bleu': 16.318658330694277}
********** Saving best result in step 5000 **********
********** Evaluating Done **********
********** Running evaluating **********
********** Step 5000 **********



100%|██████████| 42/42 [01:09<00:00,  1.49s/it]

['一种基于中心网络的战术应用场景与业务,并将网络编码技术应用于此类场景的实时数据多播业务中。在分析基于中心网络与Many-to-all业务模式特', '普适计算二维空间定位过程中,通过对三角形定位单元区域的误差分析,提出了定位单元布局(LUD)定理。在此基础上,对多个定位单元布局进行了研究,定义', '基于时序逻辑结构描述语言描述的软件体系结构']
['网络编码在实时战术数据多播中的应用', '普适计算中定位单元布局研究', '基于时序逻辑的面向方面体系结构描述语言']


100%|██████████| 42/42 [01:10<00:00,  1.68s/it]
[2022-05-29 02:10:58,545][model_v2.py][line:426][INFO] ********** Evaluating Done **********
[2022-05-29 02:10:58,546][model_v2.py][line:426][INFO] ********** Training Done **********
100%|██████████| 5000/5000 [10:14:51<00:00,  7.38s/it]

{'rouge-1': 0.4516643544176362, 'rouge-2': 0.32954916535989054, 'rouge-l': 0.40849861773713314, 'bleu': 16.318658330694277}
********** Evaluating Done **********
********** Training Done **********
